In [ ]:
import numpy as np
import pandas as pd
from  sklearn.model_selection import train_test_split
from  sklearn.linear_model import LogisticRegression
from  sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
#loading the dataset to pandas dataframe
fraud_detection = pd.read_csv('Fraud.csv')

In [ ]:
fraud_detection.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [ ]:
fraud_detection.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1048570,95,CASH_OUT,132557.35,C1179511630,479803.00,347245.65,C435674507,484329.37,616886.72,0,0
1048571,95,PAYMENT,9917.36,C1956161225,90545.00,80627.64,M668364942,0.00,0.00,0,0
1048572,95,PAYMENT,14140.05,C2037964975,20545.00,6404.95,M1355182933,0.00,0.00,0,0
1048573,95,PAYMENT,10020.05,C1633237354,90605.00,80584.95,M1964992463,0.00,0.00,0,0
1048574,95,PAYMENT,11450.03,C1264356443,80584.95,69134.92,M677577406,0.00,0.00,0,0


In [ ]:
fraud_detection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 11 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   step            1048575 non-null  int64  
 1   type            1048575 non-null  object 
 2   amount          1048575 non-null  float64
 3   nameOrig        1048575 non-null  object 
 4   oldbalanceOrg   1048575 non-null  float64
 5   newbalanceOrig  1048575 non-null  float64
 6   nameDest        1048575 non-null  object 
 7   oldbalanceDest  1048575 non-null  float64
 8   newbalanceDest  1048575 non-null  float64
 9   isFraud         1048575 non-null  int64  
 10  isFlaggedFraud  1048575 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 88.0+ MB


In [ ]:
fraud_detection.shape

(1048575, 11)

In [ ]:
fraud_detection.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [ ]:
#distribution of legit & fraud
fraud_detection['isFraud'].value_counts()

isFraud
0    1047433
1       1142
Name: count, dtype: int64

#0--> normal customer

#1 --> fraud


In [ ]:
# separating the data for analysis
legit = fraud_detection[fraud_detection.isFraud == 0]# if its 0 then the data is of that row is legit
fraud = fraud_detection[fraud_detection.isFraud == 1]#if its 1 then the data is of that row is fraud


In [ ]:
print(legit.shape)
print(fraud.shape)

(1047433, 11)
(1142, 11)


here:

110692 are the legit customer


120 are the fraud customer

In [ ]:
#stastistical measure of data
legit.amount.describe()

count    1.047433e+06
mean     1.575397e+05
std      2.541883e+05
min      1.000000e-01
25%      1.213487e+04
50%      7.621497e+04
75%      2.134928e+05
max      6.419835e+06
Name: amount, dtype: float64

In [ ]:
fraud.amount.describe()


count    1.142000e+03
mean     1.192629e+06
std      2.030599e+06
min      1.190000e+02
25%      8.607017e+04
50%      3.531794e+05
75%      1.248759e+06
max      1.000000e+07
Name: amount, dtype: float64

In [ ]:
#compare the values for both transactions
fraud_detection.groupby('isFraud').mean(numeric_only=True)# Set numeric_only to True to calculate the mean of numeric columns only

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
isFraud,,,,,,,
0,26.942944,1.575397e+05,8.736338e+05,894746.395080,978732.769117,1.114237e+06,0.0
1,48.272329,1.192629e+06,1.218636e+06,33944.321208,452866.124527,1.077940e+06,0.0


under sampling

building a sample dataset containing similar distribution  of normal and fraud customer


In [ ]:
legit_sample = legit.sample(n=492)#creating a new dataset through sampling

In [ ]:
new_dataset = pd.concat([legit_sample,fraud],axis = 0)#conating two dataframe
new_dataset.head()


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1047888,95,CASH_OUT,56745.14,C526144262,56745.14,0.0,C79051264,51433.88,108179.02,1,0
1048221,95,TRANSFER,33676.59,C732111322,33676.59,0.0,C1140210295,0.00,0.00,1,0
1048222,95,CASH_OUT,33676.59,C1000086512,33676.59,0.0,C1759363094,0.00,33676.59,1,0
1048323,95,TRANSFER,87999.25,C927181710,87999.25,0.0,C757947873,0.00,0.00,1,0
1048324,95,CASH_OUT,87999.25,C409531429,87999.25,0.0,C1827219533,0.00,87999.25,1,0


In [ ]:
new_dataset.tail()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1047888,95,CASH_OUT,56745.14,C526144262,56745.14,0.0,C79051264,51433.88,108179.02,1,0
1048221,95,TRANSFER,33676.59,C732111322,33676.59,0.0,C1140210295,0.00,0.00,1,0
1048222,95,CASH_OUT,33676.59,C1000086512,33676.59,0.0,C1759363094,0.00,33676.59,1,0
1048323,95,TRANSFER,87999.25,C927181710,87999.25,0.0,C757947873,0.00,0.00,1,0
1048324,95,CASH_OUT,87999.25,C409531429,87999.25,0.0,C1827219533,0.00,87999.25,1,0


In [ ]:
new_dataset['isFraud'].value_counts()

isFraud
1    1142
0     492
Name: count, dtype: int64

In [ ]:
new_dataset.groupby('isFraud').mean(numeric_only=True)


,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFlaggedFraud
isFraud,,,,,,,
0,25.623984,1.583523e+05,7.562826e+05,766842.567622,837529.236402,9.965994e+05,0.0
1,48.272329,1.192629e+06,1.218636e+06,33944.321208,452866.124527,1.077940e+06,0.0


In [ ]:
#splitting data into features & Targets
x = new_dataset.drop(columns='isFraud',axis=1)
y = new_dataset['isFraud']

In [ ]:
print(x)

         step      type     amount     nameOrig  oldbalanceOrg  \
317730     16   PAYMENT    8198.19   C282535541       52051.00   
913792     43  CASH_OUT  135845.40    C21260499       83516.00   
114244     11   PAYMENT   23465.44   C653533630           0.00   
570790     23   PAYMENT    2751.33   C198488410           0.00   
650050     35  CASH_OUT  183581.64   C671976989           0.00   
...       ...       ...        ...          ...            ...   
1047888    95  CASH_OUT   56745.14   C526144262       56745.14   
1048221    95  TRANSFER   33676.59   C732111322       33676.59   
1048222    95  CASH_OUT   33676.59  C1000086512       33676.59   
1048323    95  TRANSFER   87999.25   C927181710       87999.25   
1048324    95  CASH_OUT   87999.25   C409531429       87999.25   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  \
317730         43852.81    M28686927            0.00            0.00   
913792             0.00   C262355562      4416045.17      45518

In [ ]:
print(y)

317730     0
913792     0
114244     0
570790     0
650050     0
          ..
1047888    1
1048221    1
1048222    1
1048323    1
1048324    1
Name: isFraud, Length: 1634, dtype: int64


In [ ]:
#spliting the data  into training & testing data
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [ ]:
print(x.shape,x_train.shape,x_test.shape)

(1634, 10) (1307, 10) (327, 10)


In [ ]:
#model traininh
#using logistic regression
model = LogisticRegression()


In [ ]:
for column in x_train.columns:
    if x_train[column].dtype == 'object':  # Check if the column has string values
        label_encoder = LabelEncoder()
        x_train[column] = label_encoder.fit_transform(x_train[column])

In [ ]:
model.fit(x_train,y_train)

LogisticRegression()

In [ ]:
#model evalution
#cheching accuracy score
x_train_prediction = model.predict(x_train)
training_data_accuracy = accuracy_score(x_train_prediction,y_train)

In [ ]:
print('accuracy on training data:',training_data_accuracy)

accuracy on training data: 0.9173680183626626
